# Loading Dataset

You can download the dataset from {https://darwin.v7labs.com/v7-labs/covid-19-chest-x-ray-dataset?sort=priority\%3Adesc}.
The data entitled as '`darwin dataset pull v7-labs/covid-19-chest-x-ray-dataset:all-images`' will be used in this assignment. All dataset consist of 6504 images from 702 classes. We will extract the images of 4 classes (Bacterial Pneumonia, Viral Pneumonia, No Pneumonia (healthy), Covid-19) and save them as .npy file with the following code:

In [ ]:
# Imports
import warnings
warnings.simplefilter(action='ignore')
import keras
import json
import numpy as np
import glob
import pandas as pd

import urllib.request
from PIL import Image
from keras import layers, models
import tensorflow
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing



In [8]:
###################################
# > DISABLED - ALREADY EXECUTED < #
###################################

'''
Continue working in next cell
'''

# # all-images file should be uploaded to the same file
# imageNames = glob.glob("all-images/*")
# 
# dataset = []
# labels = []
# 
# for i, imName in enumerate(imageNames):
# 
#     # Opening JSON file
#     f = open(imName)
#     data = json.load(f)
#     for j in range(len(data['annotations'])):
# 
#         if 'COVID-19' in (data['annotations'][j]['name']):
#           #load images from url    
#             urllib.request.urlretrieve(data['image']['url'],"img.png")    
#             img = Image.open("img.png")
#             #convert images to grayscale
#             imgGray = img.convert('L')
#             #resize the image (156x156)
#             im = imgGray.resize((156,156), Image.LANCZOS)           
#             label = data['annotations'][j]['name']
#             dataset.append(np.array(im))
#             labels.append(label)
#             print(label)
#             break
# 
#         if 'Viral Pneumonia' in (data['annotations'][j]['name']) \
#             or 'Bacterial Pneumonia' in (data['annotations'][j]['name']) \
#             or 'No Pneumonia (healthy)' in (data['annotations'][j]['name']):
#             #load images from url    
#             urllib.request.urlretrieve(data['image']['url'],"img.png")    
#             img = Image.open("img.png")
#             #convert images to grayscale
#             imgGray = img.convert('L')
#             #resize the image (156x156)
#             im = imgGray.resize((156,156), Image.LANCZOS)           
#             label = data['annotations'][j]['name']
#             dataset.append(np.array(im))
#             labels.append(label)
#             break
# 
# #Convert data shape of (n_of_samples, width, height, 1)
# dataset = np.dstack(dataset)    
# dataset = np.rollaxis(dataset,-1)
# labels = np.array(labels)
# 
# #convert images gray scale to rgb
# data = np.array(layers.Lambda(tf.image.grayscale_to_rgb)(tf.expand_dims(dataset, -1)))
# 
# # save data and labels into a folder
# np.save("data.npy", data)
# np.save("labels.npy", labels)

'\nContinue working in next cell\n'

Once you save your data, you can load it from your directory.

In [9]:
data = np.load('data.npy')
labels = np.load('labels.npy')

# Preprocessing Steps



## Splitting Data

In [10]:
# stratify?
# shuffle?
from sklearn.model_selection import train_test_split

# Load the dataset
# Replace 'features' and 'labels' with your actual feature and label data
features = np.load('data.npy')
labels = np.load('labels.npy')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)


## Normalize Data

In [11]:



# Compute the mean and standard deviation of the training set
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

# Normalize each set separately using the training set statistics
X_train_norm = (X_train - train_mean) / train_std
X_val_norm = (X_val - train_mean) / train_std
X_test_norm = (X_test - train_mean) / train_std


In [12]:
# Define a dictionary that maps each category to a numerical value
label_map = {"Bacterial Pneumonia": 0, "Viral Pneumonia": 1, "No Pneumonia (healthy)": 2, "COVID-19": 3}

# Encode the categorical labels as numerical values using the label map
y_train_encoded = np.vectorize(label_map.get)(y_train)
y_val_encoded = np.vectorize(label_map.get)(y_val)
y_test_encoded = np.vectorize(label_map.get)(y_test)

# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical(y_val_encoded, num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)



# Create Baseline Model

In [13]:
from tensorflow import keras

model = keras.Sequential([
    # Convolutional layers
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(156, 156, 3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Dense layers
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

# Compile the model with appropriate loss function, optimizer, and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model for 10 epochs with a batch size of 32
history = model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_norm, y_val_onehot)
)




Epoch 1/10
128/128 [==============================] - 645s 5s/step - loss: 0.7968 - accuracy: 0.6746 - val_loss: 0.6137 - val_accuracy: 0.7390
Epoch 2/10
128/128 [==============================] - 536s 4s/step - loss: 0.6372 - accuracy: 0.7391 - val_loss: 0.6423 - val_accuracy: 0.7312
Epoch 3/10
128/128 [==============================] - 473s 4s/step - loss: 0.5594 - accuracy: 0.7702 - val_loss: 0.5811 - val_accuracy: 0.7586
Epoch 4/10
128/128 [==============================] - 373s 3s/step - loss: 0.5050 - accuracy: 0.7939 - val_loss: 0.5608 - val_accuracy: 0.7615
Epoch 5/10
128/128 [==============================] - 374s 3s/step - loss: 0.4573 - accuracy: 0.8144 - val_loss: 0.6046 - val_accuracy: 0.7576
Epoch 6/10
128/128 [==============================] - 369s 3s/step - loss: 0.4232 - accuracy: 0.8262 - val_loss: 0.5628 - val_accuracy: 0.7889
Epoch 7/10
128/128 [==============================] - 359s 3s/step - loss: 0.3493 - accuracy: 0.8567 - val_loss: 0.5934 - val_accuracy: 0.7634

# Analyze the performance of the baseline model

In [ ]:
import matplotlib.pyplot as plt
accuracy_train = history.history['accuracy']
accuracy_val = history.history['val_accuracy']
plt.plot(accuracy_train, label='training_accuracy')
plt.plot(accuracy_val, label='validation_accuracy')
plt.title('ACCURACY OF THE MODEL')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# Adapting/fine-tuning the network

# Transfer Learning